In [1]:
pip install catboost

  Using cached catboost-1.2.7-cp312-cp312-win_amd64.whl.metadata (1.2 kB)
   ---------------------------------------- 0.0/101.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/101.7 MB ? eta -:--:--
   ---------------------------------------- 0.3/101.7 MB ? eta -:--:--
   ---------------------------------------- 0.3/101.7 MB ? eta -:--:--
   ---------------------------------------- 0.5/101.7 MB 882.6 kB/s eta 0:01:55
   ---------------------------------------- 0.8/101.7 MB 838.9 kB/s eta 0:02:01
   ---------------------------------------- 1.0/101.7 MB 949.8 kB/s eta 0:01:46
    --------------------------------------- 1.3/101.7 MB 932.1 kB/s eta 0:01:48
    --------------------------------------- 1.3/101.7 MB 932.1 kB/s eta 0:01:48
    --------------------------------------- 1.6/101.7 MB 942.3 kB/s eta 0:01:47
    --------------------------------------- 1.8/101.7 MB 976.9 kB/s eta 0:01:43
    --------------------------------------- 2.1/101.7 MB 978.7 kB/s eta 0:01:42
  


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [33]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
import numpy as np
import joblib

In [34]:
train_dir = r'C:/Users/it shop/Downloads/Telegram Desktop/Alzheimer_s Dataset/Alzheimer_s Dataset/train'  
test_dir = r'C:/Users/it shop/Downloads/Telegram Desktop/Alzheimer_s Dataset/Alzheimer_s Dataset/test'

In [35]:
# Set the input shape
input_shape = (128, 128, 3)
batch_size = 32

In [36]:
# Use ImageDataGenerator for data augmentation
data_generator = ImageDataGenerator(rescale=1./255)

In [37]:
# Load the training images
train_generator = data_generator.flow_from_directory(
    train_dir,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

Found 5121 images belonging to 4 classes.


In [38]:
# Load the test images
test_generator = data_generator.flow_from_directory(
    test_dir,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

Found 1279 images belonging to 4 classes.


In [39]:
# Define the model architecture up to the feature extraction layer
model = Sequential([
    Conv2D(32, (3, 3), input_shape=input_shape),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3)),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(64),
    Activation('relu'),
    Dropout(0.5)
])

C:\Users\it shop\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [40]:
# Extract features from training images 
train_features = model.predict(train_generator)
train_features = train_features.reshape(train_features.shape[0], -1)  # Flatten features for classifier
train_labels = train_generator.classes

C:\Users\it shop\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


161/161 ━━━━━━━━━━━━━━━━━━━━ 29s 176ms/step


In [41]:
# Extract features from test images 
test_features = model.predict(test_generator)
test_features = test_features.reshape(test_features.shape[0], -1)
test_labels = test_generator.classes

40/40 ━━━━━━━━━━━━━━━━━━━━ 8s 188ms/step


In [42]:
# Use CatBoost classifier
cb_classifier = CatBoostClassifier(iterations=1000, learning_rate=0.1, depth=6)
cb_classifier.fit(train_features, train_labels)

0:	learn: 1.3268065	total: 245ms	remaining: 4m 4s
1:	learn: 1.2770776	total: 343ms	remaining: 2m 51s
2:	learn: 1.2313891	total: 440ms	remaining: 2m 26s
3:	learn: 1.1935958	total: 557ms	remaining: 2m 18s
4:	learn: 1.1615721	total: 642ms	remaining: 2m 7s
5:	learn: 1.1332124	total: 756ms	remaining: 2m 5s
6:	learn: 1.1081262	total: 911ms	remaining: 2m 9s
7:	learn: 1.0872363	total: 1.01s	remaining: 2m 5s
8:	learn: 1.0664498	total: 1.13s	remaining: 2m 4s
9:	learn: 1.0480836	total: 1.63s	remaining: 2m 41s
10:	learn: 1.0320908	total: 1.8s	remaining: 2m 41s
11:	learn: 1.0176376	total: 2.02s	remaining: 2m 46s
12:	learn: 1.0050640	total: 2.15s	remaining: 2m 43s
13:	learn: 0.9929896	total: 2.29s	remaining: 2m 41s
14:	learn: 0.9798426	total: 2.4s	remaining: 2m 37s
15:	learn: 0.9704378	total: 2.46s	remaining: 2m 31s
16:	learn: 0.9617316	total: 2.56s	remaining: 2m 27s
17:	learn: 0.9522241	total: 2.65s	remaining: 2m 24s
18:	learn: 0.9436815	total: 2.72s	remaining: 2m 20s
19:	learn: 0.9355044	total: 2.

In [43]:
# Make predictions on the test data
test_predictions = cb_classifier.predict(test_features)

In [44]:
# Calculate accuracy
test_accuracy = accuracy_score(test_labels, test_predictions)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

Test Accuracy: 56.37%


In [45]:
# Save the Keras CNN model as an .h5 file
model.save('cnn_feature_extractor6.h5')

In [46]:
# Save the trained CatBoost model as a .pkl file
joblib.dump(cb_classifier, 'catboost_classifier.pkl')

['catboost_classifier.pkl']

In [47]:
from keras.models import load_model
import joblib
import numpy as np
from tensorflow.keras.preprocessing import image
from catboost import CatBoostClassifier

# Load the saved models
cnn_model = load_model('cnn_feature_extractor6.h5')
catboost_classifier = joblib.load('catboost_classifier.pkl')  # Load CatBoost model

def classify_image(img_path):
    # Load and preprocess the new image
    img = image.load_img(img_path, target_size=(128, 128))  # Resize to match the CNN input
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Normalize to match the training preprocessing

    # Extract features using the CNN model
    features = cnn_model.predict(img_array)
    features = features.reshape(1, -1)  # Flatten to 1D for CatBoost

    # Classify the features with the CatBoost model
    prediction = catboost_classifier.predict(features)
    
    # Ensure prediction is an integer
    predicted_class = int(prediction[0])

    # Interpret the prediction
    class_labels = {0: 'MildDemented', 1: 'ModerateDemented', 2: 'NonDemented', 3: 'VeryMildDemented'}  # Adjust based on your dataset labels
    result = class_labels[predicted_class]
    return result

# Test the function with a new image
img_path = r'C:/Users/it shop/Downloads/Telegram Desktop/Alzheimer_s Dataset/Alzheimer_s Dataset/test/NonDemented/26 (62).jpg'
result = classify_image(img_path)
print(f"The image is classified as: {result}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 297ms/step
The image is classified as: NonDemented


C:\Windows\Temp\ipykernel_6324\4095898276.py:26: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  predicted_class = int(prediction[0])
